In [1]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes
import folium
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim
import requests
from pandas.io.json import json_normalize

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    folium-0.5.0               |             py_0          45 KB  conda-forge
    branca-0.4.0               |             py_0          26 KB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    altair-4.0.1               |             py_0         575 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.0 MB

The following NEW packages will be 

In [2]:
# The code was removed by Watson Studio for sharing.

In [3]:
df_pgh.head()

,Neighborhood,Sector,Population,Tot_Working,Tot_Units,Med_HV,Med_Inc,Land_Acres,Crimes,Af_Am,...,Latinos,Age_< 5,Age_5-19,Age_20-34,Age_35-59,Age_60-74,Age_>75,Open_Land,Latitude,Longitude
0,Allegheny Center,3,933,386,1052,136300,22793,134,71,453,...,27,39,202,257,209,164,61,68,40.4530,-80.0050
1,Allegheny West,3,462,151,355,123600,45519,90,38,84,...,13,0,0,39,315,58,50,11,40.4520,-80.0158
2,Allentown,6,2500,824,1291,42200,31909,189,159,878,...,58,92,371,603,883,360,192,35,40.4223,-79.9934
3,Arlington,7,1869,673,886,44200,27380,301,50,371,...,26,129,353,364,589,166,268,125,40.4135,-79.9710
4,Arlington Heights,7,244,72,148,64400,10265,85,19,205,...,2,10,96,40,75,8,15,37,40.4169,-79.9598


In [4]:
address = 'Pittsburgh, PA'

geolocator = Nominatim(user_agent="pgh_exp")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Pittsburgh are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Pittsburgh are 40.4416941, -79.9900861.


In [5]:
map_pgh = folium.Map(location=[latitude, longitude], zoom_start=13)

# add markers to map
for lat, lng, sector, neighborhood in zip(df_pgh['Latitude'],df_pgh['Longitude'], df_pgh['Sector'], df_pgh['Neighborhood']):
    label = '{}, Sector {}'.format(neighborhood, sector)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='black',
        fill=True,
        fill_color='yellow',
        fill_opacity=0.7,
        parse_html=False).add_to(map_pgh)  
    
map_pgh

In [6]:
CLIENT_ID = 'FJDKZYMXDRHIO4W302BOSCD0YVJ33WAC1CBJEND4O01AU5AD'
CLIENT_SECRET = '1FYAET1WSZDR24NYCTRL5ZSXAKCVTIESTI4OPAOPW0XWKRNA'
VERSION = '20180605'

In [7]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [8]:
pgh_venues = getNearbyVenues(names=df_pgh['Neighborhood'],
                                   latitudes=df_pgh['Latitude'],
                                   longitudes=df_pgh['Longitude']
                                  )

Allegheny Center
Allegheny West
Allentown
Arlington
Arlington Heights
Banksville
Bedford Dwellings
Beechview
Beltzhoover
Bloomfield
Bluff
Bon Air
Brighton Heights
Brookline
California-Kirkbride
Carrick
Central Business District
Central Lawrenceville
Central Northside
Central Oakland
Chartiers City
Crafton Heights
Crawford-Roberts
Duquesne Heights
East Allegheny
East Carnegie
East Hills
East Liberty
Elliott
Esplen
Fairywood
Fineview
Friendship
Garfield
Greenfield
Hays
Hazelwood
Highland Park
Homewood North
Homewood South
Homewood West
Knoxville
Larimer
Lincoln Place
Lincoln-Lemington-Belmar
Lower Lawrenceville
Manchester
Marshall-Shadeland
Middle Hill
Morningside
Mount Washington
Mt. Oliver
New Homestead
North Oakland
Oakwood
Overbrook
Perry North
Perry South
Point Breeze
Point Breeze North
Polish Hill
Regent Square
Ridgemont
Shadyside
Sheraden
South Oakland
South Side Flats
South Side Slopes
Spring Garden
Spring Hill-City View
Squirrel Hill North
Squirrel Hill South
St. Clair
Stanton H

In [9]:
pgh_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Allegheny Center,40.453,-80.005,Children's Museum of Pittsburgh,40.452793,-80.006569,Museum
1,Allegheny Center,40.453,-80.005,Federal Galley,40.451605,-80.006045,Food Court
2,Allegheny Center,40.453,-80.005,Park House,40.453284,-80.001504,Bar
3,Allegheny Center,40.453,-80.005,El Burro,40.455860,-80.006689,Mexican Restaurant
4,Allegheny Center,40.453,-80.005,Bistro To Go,40.453450,-80.000995,Deli / Bodega


In [10]:
pgh_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Allegheny Center,29,29,29,29,29,29
Allegheny West,23,23,23,23,23,23
Allentown,7,7,7,7,7,7
Arlington,1,1,1,1,1,1
Arlington Heights,2,2,2,2,2,2
Banksville,1,1,1,1,1,1
Bedford Dwellings,4,4,4,4,4,4
Beechview,8,8,8,8,8,8
Bloomfield,30,30,30,30,30,30


In [11]:
pgh_onehot = pd.get_dummies(pgh_venues[['Venue Category']], prefix="", prefix_sep="")
pgh_onehot['Neighborhood'] = pgh_venues['Neighborhood']
pgh_grouped = pgh_onehot.groupby('Neighborhood').mean().reset_index()
pgh_grouped

,Neighborhood,Accessories Store,Adult Boutique,American Restaurant,Aquarium,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,...,Trail,Tunnel,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Wine Bar,Yoga Studio,Zoo,Zoo Exhibit
0,Allegheny Center,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.034483,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.034483,0.0
1,Allegheny West,0.00,0.000000,0.086957,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.0
2,Allentown,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.142857,0.00,0.000000,0.00,0.000000,0.000000,0.0
3,Arlington,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.0
4,Arlington Heights,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.500000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.0
5,Banksville,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.0
6,Bedford Dwellings,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.0
7,Beechview,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.0
8,Bloomfield,0.00,0.000000,0.000000,0.0,0.000000,0.033333,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.0
9,Bluff,0.00,0.000000,0.100000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.0


In [12]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [13]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = pgh_grouped['Neighborhood']

for ind in np.arange(pgh_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(pgh_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Allegheny Center,Deli / Bodega,Exhibit,Burger Joint,Pizza Place,Bakery
1,Allegheny West,Fast Food Restaurant,American Restaurant,Sandwich Place,Pub,BBQ Joint
2,Allentown,Deli / Bodega,Diner,Vegetarian / Vegan Restaurant,Coffee Shop,Discount Store
3,Arlington,Cosmetics Shop,Electronics Store,Film Studio,Field,Fast Food Restaurant
4,Arlington Heights,Baseball Field,Trail,Zoo Exhibit,Electronics Store,Film Studio


In [14]:
pgh_merged = df_pgh

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
pgh_merged = pgh_merged.join(pgh_grouped.set_index('Neighborhood'), on='Neighborhood')

pgh_merged.head()

,Neighborhood,Sector,Population,Tot_Working,Tot_Units,Med_HV,Med_Inc,Land_Acres,Crimes,Af_Am,...,Trail,Tunnel,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Wine Bar,Yoga Studio,Zoo,Zoo Exhibit
0,Allegheny Center,3,933,386,1052,136300,22793,134,71,453,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.034483,0.0
1,Allegheny West,3,462,151,355,123600,45519,90,38,84,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0
2,Allentown,6,2500,824,1291,42200,31909,189,159,878,...,0.0,0.0,0.0,0.142857,0.0,0.0,0.0,0.0,0.000000,0.0
3,Arlington,7,1869,673,886,44200,27380,301,50,371,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0
4,Arlington Heights,7,244,72,148,64400,10265,85,19,205,...,0.5,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0


In [15]:
pgh_clust= pgh_merged.drop(['Neighborhood','Sector','Latitude','Longitude'], 1)
pgh_clust.head()

,Population,Tot_Working,Tot_Units,Med_HV,Med_Inc,Land_Acres,Crimes,Af_Am,Asians,Other,...,Trail,Tunnel,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Wine Bar,Yoga Studio,Zoo,Zoo Exhibit
0,933,386,1052,136300,22793,134,71,453,56,6,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.034483,0.0
1,462,151,355,123600,45519,90,38,84,7,3,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0
2,2500,824,1291,42200,31909,189,159,878,1,20,...,0.0,0.0,0.0,0.142857,0.0,0.0,0.0,0.0,0.000000,0.0
3,1869,673,886,44200,27380,301,50,371,3,7,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0
4,244,72,148,64400,10265,85,19,205,0,4,...,0.5,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0


In [16]:
from sklearn.preprocessing import StandardScaler

X = pgh_clust.values[:,1:]
X = np.nan_to_num(X)
pgh_norm = StandardScaler().fit_transform(X)
pgh_norm

array([[-0.74245831, -0.47114554,  0.82526031, ..., -0.1470871 ,
         4.56463763, -0.14496601],
       [-0.87628438, -0.89721391,  0.60056445, ..., -0.1470871 ,
        -0.15008816, -0.14496601],
       [-0.4930293 , -0.32504749, -0.83961212, ..., -0.1470871 ,
        -0.15008816, -0.14496601],
       ...,
       [-0.48733457, -0.69426599, -0.50522223, ..., -0.1470871 ,
        -0.15008816, -0.14496601],
       [-0.07218903, -0.19973326,  0.07155611, ..., -0.1470871 ,
        -0.15008816, -0.14496601],
       [-0.63938377, -0.70404661, -0.20267899, ..., -0.1470871 ,
        -0.15008816, -0.14496601]])

In [39]:
kclusters = 5
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(pgh_norm)
kmeans.labels_[0:10]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

In [40]:
pgh_merged=pgh_merged.drop('Cluster Labels',1)

In [41]:
pgh_merged.insert(0, 'Cluster Labels', kmeans.labels_)

In [42]:
pgh_merged

,Cluster Labels,Neighborhood,Sector,Population,Tot_Working,Tot_Units,Med_HV,Med_Inc,Land_Acres,Crimes,...,Trail,Tunnel,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Wine Bar,Yoga Studio,Zoo,Zoo Exhibit
0,1,Allegheny Center,3,933,386,1052,136300,22793,134,71,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.034483,0.0
1,1,Allegheny West,3,462,151,355,123600,45519,90,38,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.0
2,1,Allentown,6,2500,824,1291,42200,31909,189,159,...,0.000000,0.000000,0.000000,0.142857,0.00,0.000000,0.00,0.000000,0.000000,0.0
3,1,Arlington,7,1869,673,886,44200,27380,301,50,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.0
4,1,Arlington Heights,7,244,72,148,64400,10265,85,19,...,0.500000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.0
5,1,Banksville,5,4144,2506,2118,110900,55181,620,62,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.0
6,1,Bedford Dwellings,15,1202,305,746,81100,10891,115,42,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.0
7,1,Beechview,5,7974,3909,3939,72400,39896,934,193,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.0
8,1,Beltzhoover,6,1925,706,1085,36000,36917,269,83,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,1,Bloomfield,12,8442,5117,5136,92840,33604,449,453,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.0


In [43]:
import matplotlib.cm as cm
import matplotlib.colors as colors
map_pgh_clust = folium.Map(location=[latitude, longitude], zoom_start=13)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, neigh, sec, cluster in zip(pgh_merged['Latitude'], pgh_merged['Longitude'], pgh_merged['Neighborhood'], pgh_merged['Sector'], pgh_merged['Cluster Labels']):
    label = folium.Popup(str(neigh) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_pgh_clust)
       
map_pgh_clust